In [25]:
import numpy as np, pandas as pd
from astropy.table import Table

import sys  
sys.path.insert(0, '/Users/huiyang/Research/GitHub/MUWCLASS_CSCv2/codes/nway_cross_matching/')
from nway import nway_cross_matching


In [26]:

TD = pd.read_csv(f'./data/CSCv2_TD_11162022_MW_comb_cleaned.csv')

CSCv2 = Table.read('/Users/huiyang/Research/GitHub/MUWCLASS_CSCv2/codes/buildTD/data/CSCv2.vot').to_pandas()
CSCv2['name'] = '2CXO ' + CSCv2['_2CXO']

In [27]:
nway_all = pd.read_csv('/Users/huiyang/Research/GitHub/MUWCLASS_4FGL/codes/MW_xmatch/nway_all/CSCv2_TD_11162022_MW.csv')

nway_all['name'] = '2CXO '+nway_all['CSC__2CXO']

In [28]:
# Let's get phot_bp_rp_excess_factor

from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astroquery.vizier import Vizier
from astropy.table import Table


print(len(nway_all))
TD_gaia = nway_all.drop_duplicates(subset=['Gaia','GAIA_RA', 'GAIA_DEC'])[['Gaia','GAIA_RA', 'GAIA_DEC']]
TD_gaia = TD_gaia[TD_gaia['Gaia']!='-99'].reset_index(drop=True)


print(len(TD_gaia))
TD_gaia


TD_gaia['_q'] = TD_gaia.index+1

#'''
# cross-matching to Gaiadr3 catalog with phot_bp_rp_excess_factor

viz = Vizier(row_limit=-1,  timeout=5000, columns=["**", "+_r","phot_bp_rp_excess_factor"], catalog='I/355/gaiadr3')

radec = [[TD_gaia.loc[i, 'GAIA_RA'], TD_gaia.loc[i, 'GAIA_DEC']] for i in range(len(TD_gaia))]
rd = Table(Angle(radec, 'deg'), names=('_RAJ2000', '_DEJ2000'))
df_gaiadr3 = viz.query_region(rd, radius=0.1*u.arcsec)[0].to_pandas()
#'''
#print(len(df_gaiadist))
TD_gaia = pd.merge(TD_gaia, df_gaiadr3, on=['_q'], how='inner')#.rename(columns={'Flag':'gaiadist_flag'})
#print(len(TD_gaia))

#TD_MW = pd.merge(TD_MW, TD_gaia[['CSC__2CXO','rgeo','b_rgeo','B_rgeo','rpgeo','b_rpgeo','B_rpgeo','gaiadist_flag']], on='CSC__2CXO',how='outer')

TD_gaia = TD_gaia[TD_gaia['_r']<=0.0001].reset_index(drop=True)

nway_all['E_BP_RP_'] = -99

for gaia_src in TD_gaia['Gaia'].unique():
    nway_all.loc[nway_all['Gaia']==gaia_src, 'E_BP_RP_'] = TD_gaia.loc[TD_gaia['Gaia']==gaia_src, 'E_BP_RP_'].values[0]


35634
3106


In [29]:
nway_all.columns

Index(['CSC_ID', 'CSC_RA', 'CSC_DEC', 'CSC_err_r0', 'CSC_err_r1', 'CSC_PA',
       'CSC__2CXO', 'CSC__r', 'CSC_fe', 'CSC_fc',
       ...
       'dist_bayesfactor', 'dist_bayesfactor_corrected', 'dist_post',
       'p_single', 'p_any', 'p_i', 'match_flag', 'Gaia', 'name', 'E_BP_RP_'],
      dtype='object', length=106)

In [30]:
df_new1 = TD[~TD['name'].isin(nway_all['name'])].reset_index(drop=True)
df_new1_2 = CSCv2[CSCv2['name'].isin(df_new1['name'])].reset_index(drop=True)

In [31]:
len(nway_all)#.columns

35634

In [32]:
# some new sources 



add_HMXBs = ['2CXO J163905.4-464212','2CXO J081228.3-311452','2CXO J163553.8-472540','2CXO J203213.1+412724']
add_HMXBs_confused = ['2CXO J131825.0-625815','2CXO J174906.8-273233','2CXO J174912.4-272538']
add_CVs = ['2CXO J174954.6-294335']
add_new = ['2CXO J163905.4-464212','2CXO J081228.3-311452','2CXO J131825.0-625815','2CXO J174906.8-273233','2CXO J174912.4-272538','2CXO J174954.6-294335']
df_new2 = CSCv2[CSCv2['name'].isin(add_HMXBs+add_HMXBs_confused+add_CVs)].reset_index(drop=True)

PU_increase_2 =  ['2CXO J163905.4-464212','2CXO J163553.8-472540']
df_new2.loc[df_new2['name'].isin(PU_increase_2 ), 'r0'] = df_new2.loc[df_new2['name'].isin(PU_increase_2 ), 'r0']*2
df_new2.loc[df_new2['name'].isin(PU_increase_2 ), 'r1'] = df_new2.loc[df_new2['name'].isin(PU_increase_2 ), 'r1']*2

PU_decrease_2 =  ['2CXO J203213.1+412724']
df_new2.loc[df_new2['name'].isin(PU_decrease_2 ), 'r0'] = df_new2.loc[df_new2['name'].isin(PU_decrease_2 ), 'r0']/2
df_new2.loc[df_new2['name'].isin(PU_decrease_2 ), 'r1'] = df_new2.loc[df_new2['name'].isin(PU_decrease_2 ), 'r1']/2



In [33]:
df_new = pd.concat([df_new1_2, df_new2], ignore_index=True, sort=False)

In [34]:
query_dir = '/Users/huiyang/Research/GitHub/MUWCLASS_CSCv2/codes/nway_cross_matching/data/query'
nway_radius = 0.01

df_MW = pd.DataFrame()
for i in range(len(df_new)):
    #nway_cross_matching(TD, i, radius, query_dir,explain=True,rerun=True)
    csc_name = nway_cross_matching(df_new, i, nway_radius, query_dir,name_col='name',ra_col='RAICRS',dec_col='DEICRS', ra_csc_col='RAICRS',dec_csc_col='DEICRS',PU_col='r0',r0_col='r0',r1_col='r1',PA_col='PA',data_dir='./data/TD_new_nway',explain=False,move=False,move_dir='check',rerun=False, sigma=2.,newcsc=True)
    
    dat = Table.read(f'./data/TD_new_nway/{csc_name}_nway.fits', format='fits')
    df = dat.to_pandas()


    df_MW = pd.concat([df_MW, df], ignore_index=True, sort=False)

print(df_MW['match_flag'].value_counts())

str_cols = ['CSC__2CXO','TMASS__2MASS','ALLWISE_AllWISE','CATWISE_Name']
for col in str_cols:
    df_MW[col] = df_MW[col].str.decode("utf-8")

df_MW['name'] = df_MW['CSC__2CXO']
df_MW['Gaia'] = df_MW.apply(lambda r: 'Gaia DR3 '+ str(r['GAIA_Source']) if str(r['GAIA_Source'])!='-99' else '-99', axis=1)

#df_MW.to_csv(f'{output_name}.csv',index=False)


0    35
1    11
Name: match_flag, dtype: int64


In [35]:
TD_gaia = df_MW.drop_duplicates(subset=['Gaia','GAIA_RA', 'GAIA_DEC'])[['Gaia','GAIA_RA', 'GAIA_DEC']]
TD_gaia = TD_gaia[TD_gaia['Gaia']!='-99'].reset_index(drop=True)


print(len(TD_gaia))
#TD_gaia


TD_gaia['_q'] = TD_gaia.index+1

#'''
# cross-matching to Gaiadr3 catalog with phot_bp_rp_excess_factor

viz = Vizier(row_limit=-1,  timeout=5000, columns=["**", "+_r","phot_bp_rp_excess_factor"], catalog='I/355/gaiadr3')

radec = [[TD_gaia.loc[i, 'GAIA_RA'], TD_gaia.loc[i, 'GAIA_DEC']] for i in range(len(TD_gaia))]
rd = Table(Angle(radec, 'deg'), names=('_RAJ2000', '_DEJ2000'))
df_gaiadr3 = viz.query_region(rd, radius=0.1*u.arcsec)[0].to_pandas()
#'''
#print(len(df_gaiadist))
TD_gaia = pd.merge(TD_gaia, df_gaiadr3, on=['_q'], how='inner')#.rename(columns={'Flag':'gaiadist_flag'})
#print(len(TD_gaia))

TD_gaia = TD_gaia[TD_gaia['_r']<=0.0001].reset_index(drop=True)

df_MW['E_BP_RP_'] = -99

for gaia_src in TD_gaia['Gaia'].unique():
    df_MW.loc[df_MW['Gaia']==gaia_src, 'E_BP_RP_'] = TD_gaia.loc[TD_gaia['Gaia']==gaia_src, 'E_BP_RP_'].values[0]

6


In [36]:
print(len(nway_all))
print(len(nway_all.columns), len(df_MW.columns))
nway_all = nway_all[~nway_all['name'].isin(df_new['name'])]
print(len(nway_all), len(nway_all['name'].unique()))

df_MW = pd.concat([nway_all, df_MW], ignore_index=True, sort=False).reset_index(drop=True)
print(len(df_MW), len(df_MW['name'].unique()))

35634
106 103
35615 3818
35661 3829


In [37]:
df_MW.columns[:100]

Index(['CSC_ID', 'CSC_RA', 'CSC_DEC', 'CSC_err_r0', 'CSC_err_r1', 'CSC_PA',
       'CSC__2CXO', 'CSC__r', 'CSC_fe', 'CSC_fc', 'GAIA_RA', 'GAIA_DEC',
       'GAIA_PU', 'GAIA_Source', 'GAIA__r', 'GAIA_e_Pos', 'GAIA_Plx',
       'GAIA_e_Plx', 'GAIA_PM', 'GAIA_e_PM', 'GAIA_epsi', 'GAIA_Gmag',
       'GAIA_BPmag', 'GAIA_RPmag', 'GAIA_e_Gmag', 'GAIA_e_BPmag',
       'GAIA_e_RPmag', 'TMASS_RA', 'TMASS_DEC', 'TMASS_err0', 'TMASS_err1',
       'TMASS_errPA', 'TMASS__2MASS', 'TMASS__r', 'TMASS_Jmag', 'TMASS_Hmag',
       'TMASS_Kmag', 'TMASS_e_Jmag', 'TMASS_e_Hmag', 'TMASS_e_Kmag',
       'ALLWISE_RA', 'ALLWISE_DEC', 'ALLWISE_err0', 'ALLWISE_err1',
       'ALLWISE_errPA', 'ALLWISE_AllWISE', 'ALLWISE__r', 'ALLWISE_W1mag',
       'ALLWISE_W2mag', 'ALLWISE_W3mag', 'ALLWISE_W4mag', 'ALLWISE_e_W1mag',
       'ALLWISE_e_W2mag', 'ALLWISE_e_W3mag', 'ALLWISE_e_W4mag', 'CATWISE_RA',
       'CATWISE_DEC', 'CATWISE_PU', 'CATWISE_Name', 'CATWISE__r',
       'CATWISE_W1mproPM', 'CATWISE_W2mproPM', 'CATWISE_e_

In [38]:
print(len(nway_all))
TD_catalog = df_MW.drop_duplicates(subset=['ALLWISE_AllWISE','ALLWISE_RA','ALLWISE_DEC'])[['ALLWISE_AllWISE','ALLWISE_RA','ALLWISE_DEC']]
TD_catalog = TD_catalog[TD_catalog['ALLWISE_AllWISE']!='-99'].reset_index(drop=True)


print(len(TD_catalog))



TD_catalog['_q'] = TD_catalog.index+1

#'''
# cross-matching to Gaiadr3 catalog with phot_bp_rp_excess_factor

viz = Vizier(row_limit=-1,  timeout=5000, columns=["**", "+_r"], catalog='II/328/allwise')

radec = [[TD_catalog.loc[i, 'ALLWISE_RA'], TD_catalog.loc[i, 'ALLWISE_DEC']] for i in range(len(TD_catalog))]
rd = Table(Angle(radec, 'deg'), names=('_RAJ2000', '_DEJ2000'))
df_catalog = viz.query_region(rd, radius=0.1*u.arcsec)[0].to_pandas()
#'''
print(len(TD_catalog))
TD_catalog = pd.merge(TD_catalog, df_catalog, on=['_q'], how='inner')#.rename(columns={'Flag':'gaiadist_flag'})

TD_catalog = TD_catalog[TD_catalog['_r']<=0.0001].reset_index(drop=True)
print(len(TD_catalog))


flag_cols = ['na', 'sat1', 'sat2', 'sat3', 'ccf', 'ex', 'qph']
for flag in flag_cols:
    df_MW[flag] = -99
    for cat_src in TD_catalog['ALLWISE_AllWISE'].unique():
        
        df_MW.loc[df_MW['ALLWISE_AllWISE']==cat_src, flag] = TD_catalog.loc[TD_catalog['ALLWISE_AllWISE']==cat_src, flag].values[0]


35615
2455


2455
2455


In [41]:
print(len(nway_all))
TD_catalog = df_MW.drop_duplicates(subset=['TMASS__2MASS','TMASS_RA', 'TMASS_DEC'])[['TMASS__2MASS','TMASS_RA', 'TMASS_DEC']]
TD_catalog = TD_catalog[TD_catalog['TMASS__2MASS']!='-99'].reset_index(drop=True)


print(len(TD_catalog))



TD_catalog['_q'] = TD_catalog.index+1

#'''
# cross-matching to Gaiadr3 catalog with phot_bp_rp_excess_factor

viz = Vizier(row_limit=-1,  timeout=5000, columns=["**", "+_r"], catalog='II/246/out')

radec = [[TD_catalog.loc[i, 'TMASS_RA'], TD_catalog.loc[i, 'TMASS_DEC']] for i in range(len(TD_catalog))]
rd = Table(Angle(radec, 'deg'), names=('_RAJ2000', '_DEJ2000'))
df_catalog = viz.query_region(rd, radius=0.1*u.arcsec)[0].to_pandas()
#'''
print(len(TD_catalog))
TD_catalog = pd.merge(TD_catalog, df_catalog, on=['_q'], how='inner')#.rename(columns={'Flag':'gaiadist_flag'})

TD_catalog = TD_catalog[TD_catalog['_r']<=0.0001].reset_index(drop=True)
print(len(TD_catalog))


#'''
flag_cols = ['Qflg', 'Rflg', 'Bflg', 'Cflg','Xflg', 'Aflg']
for flag in flag_cols:
    df_MW[flag] = -99
    for cat_src in TD_catalog['TMASS__2MASS'].unique():
        
        df_MW.loc[df_MW['TMASS__2MASS']==cat_src, flag] = TD_catalog.loc[TD_catalog['TMASS__2MASS']==cat_src, flag].values[0]
#'''

35615
2386
2386
2386


In [40]:
TD_catalog.columns



Index(['TMASS__2MASS', 'TMASS_RA', 'TMASS_DEC', '_q', '_r', 'RAJ2000',
       'DEJ2000', 'errMaj', 'errMin', 'errPA', '_2MASS', 'Jmag', 'Jcmsig',
       'e_Jmag', 'Jsnr', 'Hmag', 'Hcmsig', 'e_Hmag', 'Hsnr', 'Kmag', 'Kcmsig',
       'e_Kmag', 'Ksnr', 'Qflg', 'Rflg', 'Bflg', 'Cflg', 'Ndet', 'prox',
       'pxPA', 'pxCntr', 'Xflg', 'Aflg', 'Cntr', 'Hemis', 'Date', 'Scan',
       'GLON', 'GLAT', 'Xscan', '_tab1_36', 'Jpsfchi', 'Hpsfchi', 'Kpsfchi',
       'Jstdap', 'e_Jstdap', 'Hstdap', 'e_Hstdap', 'Kstdap', 'e_Kstdap',
       'edgeNS', 'edgeEW', 'edge', 'dup', 'use', 'opt', 'Dopt', 'PAopt',
       'Bmag', 'Rmag', 'Nopt', 'extKey', 'scanKey', 'coaddKey', 'coadd',
       'Opt'],
      dtype='object')

In [43]:
catalogs = {'GAIA': ['Gmag','BPmag','RPmag'], 
            'TMASS': ['Jmag','Hmag','Kmag' ],
            'ALLWISE': ['W1mag','W2mag','W3mag','W4mag'],
            'CATWISE':['W1mproPM','W2mproPM']}
for cat in catalogs:
    for col in catalogs[cat]:
        #print(col)
        df_MW = df_MW.rename(columns={cat+'_'+col:col, cat+'_e_'+col:'e_'+col})

df_MW = df_MW.fillna('-99')
df_MW = df_MW.replace({-99:'-99'})
df_MW = df_MW.replace({-99.:'-99'})
#df_comb = df_comb.replace({'-99':np.nan})
df_MW = df_MW.replace({1e20:'-99'})

# combine ALLWISE, CatWISE and UnWISE

for w in ['W1','W2']:
    df_MW.loc[df_MW['ALLWISE_AllWISE']=='-99', w+'mag']      = df_MW.loc[df_MW['ALLWISE_AllWISE']=='-99', w+'mproPM']
    df_MW.loc[df_MW['ALLWISE_AllWISE']=='-99', 'e_'+w+'mag'] = df_MW.loc[df_MW['ALLWISE_AllWISE']=='-99', 'e_'+w+'mproPM']

df_MW = df_MW.replace({'-99':np.nan})



In [44]:
CGS = pd.read_csv('/Users/huiyang/Research/GitHub/MUWCLASS_CSCv2/old_files/CSCv2_all_MW_classprob_XCLASS.csv')

TD_new = CGS[CGS['name'].isin(add_new)].reset_index(drop=True)



In [45]:
TD_new_class = pd.DataFrame.from_dict({'Class':['HMXB','HMXB','HMXB','HMXB','HMXB','CV'], 
         'name':['2CXO J081228.3-311452', '2CXO J131825.0-625815',
               '2CXO J163905.4-464212', '2CXO J174906.8-273233',
               '2CXO J174912.4-272538', '2CXO J174954.6-294335']})

TD_new = pd.merge(TD_new, TD_new_class, on='name').rename(columns={'Class_y':'Class'})

In [46]:
TD = pd.concat([TD[['name','ra', 'dec','flux_aper90_ave_s', 'e_flux_aper90_ave_s',
       'flux_aper90_ave_m', 'e_flux_aper90_ave_m', 'flux_aper90_ave_h','powlaw_gamma_mean','e_powlaw_gamma_mean',
       'e_flux_aper90_ave_h','kp_prob_b_max','var_inter_prob','Class']].rename(columns = {
       'flux_aper90_ave_s':'Fcsc_s','e_flux_aper90_ave_s':'e_Fcsc_s',
       'flux_aper90_ave_m':'Fcsc_m','e_flux_aper90_ave_m':'e_Fcsc_m',
       'flux_aper90_ave_h':'Fcsc_h','e_flux_aper90_ave_h':'e_Fcsc_h','kp_prob_b_max':'var_intra_prob'}), 
                TD_new[['name','ra', 'dec','F_s', 'e_Fcsc_s',
       'F_m', 'e_Fcsc_m', 'F_h', 'e_Fcsc_h','P_intra', 'P_inter','Class']].rename(columns={
       'F_s':'Fcsc_s','F_m':'Fcsc_m','F_h':'Fcsc_h',
       'P_intra':'var_intra_prob','P_inter':'var_inter_prob'}) ], ignore_index=True, sort=False)

In [47]:
len(TD['name'].unique()), len(df_MW['name'].unique())

print(len(df_MW))
df_MW = df_MW[df_MW['name'].isin(TD['name'])].reset_index(drop=True)
print(len(df_MW))

len(df_MW.loc[df_MW['p_any']==0, 'name'].unique()), len(df_MW[df_MW['p_any']==0])

df_MW.loc[df_MW['p_any']==0, 'p_i'] = 1.
df_MW.loc[(df_MW['ncat']==1) & (df_MW['p_any']!=0), 'match_flag'] = 3
#df = recal_matchflag(df, ratio=2., csc='CSC__2CXO')
print(df_MW['match_flag'].value_counts())
df_MW.loc[df_MW['match_flag']==3, 'p_i'] = 1.- df_MW.loc[df_MW['match_flag']==3, 'p_any']
df_MW.loc[(df_MW['match_flag']!=3) & (df_MW['p_any']!=0), 'p_i'] = df_MW.loc[(df_MW['match_flag']!=3) & (df_MW['p_any']!=0), 'p_i'] * df_MW.loc[(df_MW['match_flag']!=3) & (df_MW['p_any']!=0), 'p_any']
df_MW = df_MW.sort_values(by=['name','p_i'],ascending=[True, False]).reset_index(drop=True)

35661
30212
0    24046
1     3132
3     2881
2      153
Name: match_flag, dtype: int64


In [48]:
df_MW['p_l'], df_MW['p_u']= np.nan, np.nan

for name in df_MW['name'].unique():
    idxs = df_MW[df_MW['name']==name].index
    #print(idxs)
    p_s = 0.
    for idx in idxs:
        df_MW.loc[idx, 'p_l'] = p_s
        p_s += df_MW.loc[idx, 'p_i']
        df_MW.loc[idx, 'p_u'] = p_s
        

In [51]:
df_MW.columns[:100]

Index(['CSC_ID', 'CSC_RA', 'CSC_DEC', 'CSC_err_r0', 'CSC_err_r1', 'CSC_PA',
       'CSC__2CXO', 'CSC__r', 'CSC_fe', 'CSC_fc', 'GAIA_RA', 'GAIA_DEC',
       'GAIA_PU', 'GAIA_Source', 'GAIA__r', 'GAIA_e_Pos', 'GAIA_Plx',
       'GAIA_e_Plx', 'GAIA_PM', 'GAIA_e_PM', 'GAIA_epsi', 'Gmag', 'BPmag',
       'RPmag', 'e_Gmag', 'e_BPmag', 'e_RPmag', 'TMASS_RA', 'TMASS_DEC',
       'TMASS_err0', 'TMASS_err1', 'TMASS_errPA', 'TMASS__2MASS', 'TMASS__r',
       'Jmag', 'Hmag', 'Kmag', 'e_Jmag', 'e_Hmag', 'e_Kmag', 'ALLWISE_RA',
       'ALLWISE_DEC', 'ALLWISE_err0', 'ALLWISE_err1', 'ALLWISE_errPA',
       'ALLWISE_AllWISE', 'ALLWISE__r', 'W1mag', 'W2mag', 'W3mag', 'W4mag',
       'e_W1mag', 'e_W2mag', 'e_W3mag', 'e_W4mag', 'CATWISE_RA', 'CATWISE_DEC',
       'CATWISE_PU', 'CATWISE_Name', 'CATWISE__r', 'W1mproPM', 'W2mproPM',
       'e_W1mproPM', 'e_W2mproPM', 'Separation_GAIA_CSC',
       'Separation_GAIA_CSC_ra', 'Separation_GAIA_CSC_dec',
       'Separation_TMASS_CSC', 'Separation_TMASS_CSC_ra',
  

In [52]:
df_MW[['name','Gmag', 'BPmag', 'RPmag', 'e_Gmag', 'e_BPmag', 'e_RPmag','E_BP_RP_','Jmag', 'Hmag', 'Kmag', 'e_Jmag', 'e_Hmag','Qflg','Rflg','Bflg','Cflg','Xflg','Aflg',
       'e_Kmag','W1mag','W2mag','W3mag', 'W4mag', 'e_W1mag', 'e_W2mag', 'e_W3mag', 'e_W4mag','na', 'sat1', 'sat2', 'sat3', 'ccf', 'ex', 'qph',
       'ncat','p_any','p_i','p_l','p_u','match_flag','Gaia','TMASS__2MASS','ALLWISE_AllWISE','CATWISE_Name']].to_csv('./data/TD_v03022023/TD_v03022023_MW.csv', index=False)

TD.to_csv('./data/TD_v03022023/TD_v03022023_Xray.csv', index=False)

In [291]:
TD['Class'].value_counts()

AGN        1405
YSO         748
LM-STAR     607
HM-STAR     128
NS           95
LMXB         63
CV           48
HMXB         38
Name: Class, dtype: int64

In [358]:
len(TD['name'].unique()), len(df_MW['name'].unique())

(3132, 3132)

In [359]:
len(TD[TD['name'].isin(df_MW['name'])])

3132

In [360]:
TD

,name,ra,dec,Fcsc_s,e_Fcsc_s,Fcsc_m,e_Fcsc_m,Fcsc_h,powlaw_gamma_mean,e_powlaw_gamma_mean,e_Fcsc_h,var_intra_prob,var_inter_prob,Class
0,2CXO J000150.5+233015,0.460542,23.504339,2.271000e-15,6.440000e-16,1.335000e-16,1.335000e-16,7.978846e-18,NaN,NaN,6.028103e-18,NaN,NaN,LM-STAR
1,2CXO J000701.5+730308,1.756542,73.052258,6.539798e-15,7.795001e-16,5.389000e-15,8.090000e-16,2.488798e-14,NaN,NaN,3.525002e-15,0.732,NaN,NS
2,2CXO J001826.4+300400,4.610042,30.066714,1.096282e-15,7.074501e-16,3.585718e-16,3.096550e-16,1.314160e-15,NaN,NaN,1.134900e-15,0.838,NaN,LM-STAR
3,2CXO J005240.0+562837,13.166792,56.477003,6.646382e-15,8.588079e-16,1.007737e-15,2.834520e-16,1.801145e-15,NaN,NaN,8.182476e-16,0.934,0.120,LM-STAR
4,2CXO J010705.6+321056,16.773708,32.182486,3.128202e-14,5.515001e-15,6.775798e-15,2.742500e-15,7.669114e-15,NaN,NaN,5.039164e-15,0.933,NaN,LM-STAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3127,2CXO J174954.6-294335,267.477583,-29.726492,2.346287e-15,1.053659e-15,8.862719e-15,2.016919e-15,4.825450e-13,NaN,NaN,3.145284e-14,1.000,0.999,CV
3128,2CXO J131825.0-625815,199.604417,-62.970975,7.978846e-18,6.028103e-18,7.978846e-18,6.028103e-18,1.659042e-13,NaN,NaN,3.600630e-14,NaN,NaN,HMXB
3129,2CXO J163905.4-464212,249.772792,-46.703497,4.506595e-15,2.133531e-15,5.298133e-15,9.379576e-16,9.790126e-12,NaN,NaN,1.013930e-13,1.000,1.000,HMXB
3130,2CXO J174912.4-272538,267.301667,-27.427286,1.526036e-15,1.068777e-15,5.972962e-15,2.960973e-15,1.615903e-12,NaN,NaN,1.082808e-13,0.985,0.995,HMXB


In [312]:

df_X = pd.read_csv(f'./data/TD_v03022023/TD_v03022023_Xray.csv')
df_MW = pd.read_csv(f'./data/TD_v03022023/TD_v03022023_MW.csv')

np.random.seed(31)
p_nways = np.random.uniform(low=0.0, high=1.0, size=len(df_X))

df_MW_sel = pd.DataFrame()
for name, p_nway in zip(df_MW['name'].unique(), p_nways):
        df_MW_sel = pd.concat([df_MW_sel, df_MW[(df_MW['name']==name) & (df_MW['p_l']<=p_nway) &(df_MW['p_u']>p_nway)]], ignore_index=True, sort=False)
        #print(df_MW[(df_MW['name']==name) & (df_MW['p_l']<=p_nway) &(df_MW['p_u']>p_nway)])
        

In [311]:
match0 = df_MW_sel.loc[df_MW_sel['match_flag']==0, 'name'].values

In [313]:
match1 = df_MW_sel.loc[df_MW_sel['match_flag']==0, 'name'].values

In [320]:
len([word for word in match0 if any(digram in word for digram in match1)])

[word for word in match0 if any(digram in word for digram in match1)]

#len(match0)

['2CXO J015650.2+005308',
 '2CXO J015751.4+375305',
 '2CXO J015841.4-432503',
 '2CXO J024002.4+610744',
 '2CXO J025642.5-331521',
 '2CXO J043646.8+293832',
 '2CXO J053516.9-054555',
 '2CXO J053520.2-051359',
 '2CXO J053520.3-053039',
 '2CXO J054152.1-015717',
 '2CXO J095806.2+020113',
 '2CXO J100012.9+023522',
 '2CXO J102332.1+633508',
 '2CXO J103715.3+465014',
 '2CXO J111438.6-611116',
 '2CXO J122546.7+123943',
 '2CXO J124850.7-412654',
 '2CXO J141249.0-402136',
 '2CXO J150243.0+111557',
 '2CXO J150903.5-584821',
 '2CXO J160643.2-530142',
 '2CXO J173527.5-325554',
 '2CXO J193621.7+293455',
 '2CXO J195408.1+442216',
 '2CXO J201949.1+405209',
 '2CXO J203238.3+412857',
 '2CXO J203318.2+405610',
 '2CXO J203326.1+411549',
 '2CXO J214229.3+123317',
 '2CXO J222602.1+653146',
 '2CXO J230631.0+622831']

In [303]:
df_MW_sel['match_flag'].value_counts()

1    2751
3     180
0     154
2      47
Name: match_flag, dtype: int64

In [305]:
df_MW_sel['match_flag'].value_counts()

1    2726
3     182
0     179
2      45
Name: match_flag, dtype: int64